# 小学校の在籍人数を地図上にマッピングする（Folium、GeoPandas利用）

## はじめに
子どもの数が減ってきて、学校の統廃合が現実に議論されるようになってきました。
また、公共施設の在り方（取り壊す、残す）についても議論が及んできています。
そこで、各校区の在籍人数をマッピングをしたいと思い、試行錯誤して何とか表示できた過程の記録として残します。

## 行いたいこと
（1）学校の位置を地図上に示す
→経度緯度と学校名のリストがあればよい

（2）学校の校区を示す
→ポリゴン操作が必要

（3）各学校の在籍人数を示す
→階層プロットが必要

なお対象地域は、埼玉県久喜市とします。

### 環境
Win10
Anaconda
python3.7
モジュール
Folium、geopandas

## GISデータを取り扱う上での最低限のイメージ



### GEOデータマッピングで必要な知識
[ジオメトリタイプ一覧をながめる](https://qiita.com/boiledorange73/items/6d1200b69b5d5c88bcd0)
マッピングの基本種類
単一
POINT
    単一の点。
LINESTRING
    単一の折れ線。
POLYGON
    単一のポリゴン。リングは複数あっても良い。
POLYHEDRALSURFACE
    多面体サーフェス。
マルチ
MULTIPOINT
    複数の点からなるジオメトリ
MULTILINESTRING
    複数の折れ線からなるジオメトリ
MULTIPOLYGON
    複数のポリゴンからなるジオメトリ


### データをマッピングする方法（ソフト）
・QGIS（GISソフト）
・Folium（Python、OpenStreetMap）
（Folium は javascript ライブラリの leafletを、Python で使えるようにしたモジュール。）
など
[GISエンジンおよびサービスのまとめ](https://www.gis-py.com/entry/2018/10/16/170338)
に詳しく書かれている

### データの入手

#### ベースになる地図（日本）
Open Street Map というオープンライセンスのソフト（QGIS3,Foliumではデフォルトで使えます。）

[基盤地図情報サイト国土地理院](https://www.gsi.go.jp/kiban/)
必要な地図をダウンロードする。

#### 境界データ
国土交通省が提供している[国土数値情報 ダウンロードサービス](http://nlftp.mlit.go.jp/ksj/)からダウンロード

#### ファイル形式
GeoJSON形式
Shape形式
CSV形式（データが入っているもの）


### データの前処理

#### データの整形ソフト・モジュール
Excel
Pandas(python)
GeoPandas(python)

GeoPandasの使い方
[geopandasの使い方をマスターしよう　～Shapeファイルの読込・作成、GeoDataFrameの扱い方まで～](https://www.gis-py.com/entry/geopandas-master)
[geopandasを使ってShapeファイルを作成しよう！ ～Airbnbのデータを可視化してみよう～](https://www.gis-py.com/entry/geopandas_shape)

geopandasやそれ以外のツールやファイル変換など
[Pythonを用いたshapefileやgeojsonの読込および描画](https://qiita.com/HidKamiya/items/5e7240f8f66c9af8b10e)

### Foliumのドキュメント
[Folium 0.10.1 documentation ](https://python-visualization.github.io/folium/index.html)

#### 必要なモジュールのインストールはこちらを参照してください。
anacondaに入っているモジュールの他に、
folium、geopandasなど

[folium（位置情報の可視化）の使い方メモ](https://qiita.com/hotoku/items/0d9ddbd24568c1a07f9f)
[Folium: Python で地図可視化](https://takaishikawa42.hatenablog.com/entry/2019/01/11/234716)
[【folium】Pythonで位置情報の可視化](https://www.st-hakky-blog.com/entry/2019/04/22/180619)

### すごく参考になった記事

[python folium で、都内の公園にまつわる情報を地図上に描画する](https://www.monotalk.xyz/blog/python-folium-%E3%81%A7%E9%83%BD%E5%86%85%E3%81%AE%E5%85%AC%E5%9C%92%E3%81%AB%E3%81%BE%E3%81%A4%E3%82%8F%E3%82%8B%E6%83%85%E5%A0%B1%E3%82%92%E5%9C%B0%E5%9B%B3%E4%B8%8A%E3%81%AB%E6%8F%8F%E7%94%BB%E3%81%99%E3%82%8B/)

[python folium を使い、都道府県の夫婦年齢差をプロットする](https://www.monotalk.xyz/blog/python-folinum-%E3%82%92%E4%BD%BF%E3%81%84%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E3%81%AE%E5%A4%AB%E5%A9%A6%E5%B9%B4%E9%BD%A2%E5%B7%AE%E3%82%92%E3%83%97%E3%83%AD%E3%83%83%E3%83%88%E3%81%99%E3%82%8B/)

###　Folium

## (0) Foliumで地図を表示してみます。
埼玉県久喜市を例にします。
久喜市の市役所：大字下早見８５－３　
緯度・経度 36.06205900,139.66683800　

In [1]:
#必要なモジュールのインポート
import json
from pathlib import Path

import folium
from folium import plugins
print( "folium version is {}".format(folium.__version__) )

import geopandas as gpd
import pandas as pd

import matplotlib.pyplot as plt

folium version is 0.10.1


In [2]:
# ベースマップの作製
map_center = [36.06205900,139.66683800] #久喜市に設定
m_kuki = folium.Map(location=map_center, tiles='openstreetmap', zoom_start=13)
m_kuki.save('kuki.html')

[【珈琲たいむ】Pythonでコロプレス図を描く](https://akatak.hatenadiary.jp/entry/2018/08/26/093839)
※コロプレス図 (choropleth map)日本語では「階級区分図」
に記載されている内容

locationで地図の中心を指定、
zoom_startで縮尺のレベルを指定（デフォルト値は10）、
tilesでベースマップの種類を指定します(デフォルトはopenstreetmap）。なお、コロプレス図が見やすいのはcartodbpositron。

foliumにビルトインされているベースマップのうち、API_keyがなくても利用できるのは以下の8つ。
openstreetmap,mapboxbright,cartodbdark_matter,cartodbpositron,mapboxcontrolroom,stamenterrain,stamentoner,stamenwatercolor

## （1）小学校の位置をプロット

[Open Data Saitama](https://opendata.pref.saitama.lg.jp)
にて検索して
組織 久喜市 【久喜市】公共施設情報 【H30】公共施設情報 のCSVファイルを取得します。

https://opendata.pref.saitama.lg.jp/data/dataset/0703b822-dee1-4cf5-943b-7cbc837108ba/resource/f200c959-b425-4d1f-81cb-75c1780dd2fa/download/inet-file01.inet-kuki.localpublic01010500.votiro02011001h30sisetu-kuki.csv

このデータには、久喜市の公共施設の住所、名称、経度緯度などの情報が記載されています。

※ダウンロードしたファイル名が長いので、「kukikokyoshisetsu.csv」に変更します。


In [3]:
# 日本語CSVファイルの読み込みのために engine = "python"としています。
df_kuki = pd.read_csv("kukikokyoshisetsu.csv", engine = "python")

In [4]:
df_kuki.head()

,識別情報,団体コード,団体名,種別,名称,概要,通称,住所_表記,施設_緯度,施設_経度,施設_電話番号,ホームページ,開始時間,終了時間,日時備考,自由記述欄
0,1,112321,久喜市,庁舎等,久喜市役所（本庁舎）,NaN,NaN,下早見85-3,36.062145,139.666817,22-1111,NaN,NaN,NaN,NaN,NaN
1,2,112321,久喜市,庁舎等,久喜市役所（第二庁舎）,NaN,NaN,北青柳1404-7,36.052625,139.659255,22-1111,NaN,NaN,NaN,NaN,NaN
2,3,112321,久喜市,庁舎等,菖蒲総合支所,NaN,NaN,菖蒲町新堀38,36.059249,139.601499,85-1111,NaN,NaN,NaN,NaN,NaN
3,4,112321,久喜市,庁舎等,栗橋総合支所,NaN,NaN,間鎌251-1,36.126453,139.696062,53-1111,NaN,NaN,NaN,NaN,NaN
4,5,112321,久喜市,庁舎等,鷲宮総合支所,NaN,NaN,鷲宮6-1-1,36.099935,139.667466,58-1111,NaN,NaN,NaN,NaN,NaN


In [5]:
#元のデータから学校関係を抽出します。
df_school=df_kuki[df_kuki["種別"]=="小中学校・給食センター"]

In [ ]:
df_school

In [6]:
# 小学校だけ抽出
df_esh=df_school.loc[117:139]

In [ ]:
df_esh.head()

In [7]:
df_esh.info()
#久喜市に小学校は23校あります。

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 117 to 139
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   識別情報     23 non-null     int64  
 1   団体コード    23 non-null     int64  
 2   団体名      23 non-null     object 
 3   種別       23 non-null     object 
 4   名称       23 non-null     object 
 5   概要       0 non-null      float64
 6   通称       0 non-null      float64
 7   住所_表記    23 non-null     object 
 8   施設_緯度    23 non-null     float64
 9   施設_経度    23 non-null     float64
 10  施設_電話番号  23 non-null     object 
 11  ホームページ   0 non-null      float64
 12  開始時間     0 non-null      float64
 13  終了時間     0 non-null      float64
 14  日時備考     0 non-null      float64
 15  自由記述欄    0 non-null      float64
dtypes: float64(9), int64(2), object(5)
memory usage: 3.1+ KB


In [8]:
#地図に位置をマッピングします。
map_center = [36.06205900,139.66683800] #久喜市に設定
m2_kuki = folium.Map(location=map_center, tiles='openstreetmap', zoom_start=13)
#Forで回します。
for i, dt in df_esh.iterrows():
    folium.Marker(location=[dt['施設_緯度'],dt['施設_経度']], popup='{},{}'.format(dt['名称'],dt['施設_電話番号'])).add_to(m2_kuki)
#     print(dt['施設_緯度'], dt['施設_経度'], dt['名称'])
m2_kuki.save('kuki_e_school.html')

## （2）久喜市の校区をプロット

#### データの入手先
[国土数値情報　小学校区データ](http://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-A27-v2_1.html)
埼玉県を指定

データ形式：
①「GML形式＝高度な内容を表現できる世界標準規格。普及はこれから
 
②「シェープファイル形式」＝現在の殆どのＧＩＳソフトで読み書きでき普及している規格

このデータは上記の形式で提供されています。
データの属性

範囲、通学区域の範囲面型（GM_Surface）

市区町村コード(A27_005)：小学校が属する行政コード、コードリスト「行政コード」　
設置主体(A27_006)：小学校の設置主体、文字列型(CharacterString)　
名称(A27_007)：小学校の名称、文字列型(CharacterString)　
所在地(A27_008)：小学校の設置所在地、文字列型(CharacterString)　

In [9]:
#GeopandasでShapeファイルを読み込む
# ファイル読込（encoding='SHIFT-JIS'とすることで日本語データにも対応）
shdf = gpd.read_file('./A27-16_11_GML/shape/A27-16_11.shp',encoding='SHIFT-JIS')

In [ ]:
shdf

In [10]:
#久喜市だけ取り出します。
#A27_005には市町村コード番号が入っているので、この値でセレクトしてもよい。
kuki_esh=shdf[shdf['A27_006']=='久喜市立']

In [11]:
#この後の処理のために学校を識別するためにidをつけます。ここではid2としています。
#　このままだと一番左のインデックス番号がID番号になります。
kuki_esh['id2']=[x for x in range(0,23)]

C:\Users\MtM\Anaconda3\envs\geo\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
#　id2がつきました。
kuki_esh

In [12]:
kuki_esh.info()
#23校あります。

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23 entries, 412 to 434
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   A27_005   23 non-null     object  
 1   A27_006   23 non-null     object  
 2   A27_007   23 non-null     object  
 3   A27_008   23 non-null     object  
 4   geometry  23 non-null     geometry
 5   id2       23 non-null     int64   
dtypes: geometry(1), int64(1), object(4)
memory usage: 1.3+ KB


In [13]:
# id2は数値ではなく文字列にしておく必要があるため型変換をしています。
kuki_esh['id2'] =  kuki_esh['id2'].astype('str')

C:\Users\MtM\Anaconda3\envs\geo\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [14]:
map_center = [36.06205900,139.66683800] #久喜市に設定
m_kuki_sh = folium.Map(location=map_center, tiles='cartodbpositron', zoom_start=13)
folium.Choropleth(geo_data=kuki_esh.to_json()).add_to(m_kuki_sh)
for i, dt in df_esh.iterrows():
    folium.Marker(location=[dt['施設_緯度'],dt['施設_経度']], popup='{}'.format(dt['名称'])).add_to(m_kuki_sh)

folium.LayerControl().add_to(m_kuki_sh)

m_kuki_sh.save('kuki_e_sh_koku.html')

geo_data=kuki_esh.to_json() dfのデータをJSON形式にして渡します。
※この後詳しく解説

## （3）各小学校の在籍者をプロット

各小学校の在籍者のデータを取ってきます。
[教育要覧（久喜市の教育）](https://www.city.kuki.lg.jp/kosodate/kyoikugyosei/kyoikuiinkai/gaiyo/kyoikuyoran.html)
このHPにある「令和元年度久喜市の教育」
分割版2（学校教育、13ページから38ページまで）（PDF：30,337KB）
をダウンロードします。
この19ページ目に年度による生徒数がまとめられています。
これを数値化します。私がやったのは、PDFをWebサービスでWordファイルにして、そのあといろいろやって、excelのCSVファイルにしました。
前処理したファイルを「児童生徒数推移.csv」としてGithubに置いておきます。
なお先ほど、境界データで付加したIDと同じ番号を同じ小学校に付加しています。

In [15]:
df_num=pd.read_csv('./児童生徒数推移.csv', engine = "python")

In [16]:
#　この後にgeojsonのデータと結びつけます。
#　geojsonの値は、基本文字列なので、その文字列と結びつけるためにIDをIntからStringに変換しています。
df_num['id'] = df_num['id'].astype('str')

In [17]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   school   23 non-null     object
 1   id       23 non-null     object
 2   H22      23 non-null     int64 
 3   H23      23 non-null     int64 
 4   H24      23 non-null     int64 
 5   H25      23 non-null     int64 
 6   H26      23 non-null     int64 
 7   H27      23 non-null     int64 
 8   H28      23 non-null     int64 
 9   H29      23 non-null     int64 
 10  H30      23 non-null     int64 
 11  R01      23 non-null     int64 
dtypes: int64(10), object(2)
memory usage: 2.3+ KB


In [ ]:
df_num

In [ ]:
# 校区の境界データをJSON化するとこのようなデータになります。
#　新たに付加した「id2」は、"feature properties id2"と各学校に入っています。
#　geopandasで読み込んだデータは".to_json()"で簡単にJSONファイルに変換することができます。
kuki_esh.to_json()

In [18]:
#　最新の令和元年度のマップを作ってみます。
map_center = [36.06205900,139.66683800] #久喜市に設定
m_kuki_sh = folium.Map(location=map_center, tiles='cartodbpositron', zoom_start=13)
folium.Choropleth(geo_data=kuki_esh.to_json()).add_to(m_kuki_sh)
m_kuki_sh.choropleth(geo_data=kuki_esh.to_json(),data=df_num,
                     columns=['id', 'R01'],
                     key_on='feature.properties.id2',
                     fill_color='OrRd',
                     threshold_scale=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900],
                     fill_opacity=0.7,
                     line_opacity=0.2,
                     legend_name='生徒数'
                    )
# folium.LayerControl().add_to(m_kuki_sh)
for i, dt in df_esh.iterrows():
    folium.Marker(location=[dt['施設_緯度'],dt['施設_経度']], popup='{}'.format(dt['名称'])).add_to(m_kuki_sh)
# m_saitama
m_kuki_sh.save('kuki_esh_r1.html')

C:\Users\MtM\Anaconda3\envs\geo\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


#### fill_color　を変更することで色を選べます。
##### Colerのサンプル
[color mapの一覧をheatmapで(160個くらい画像があるので注意)](https://pod.hatenablog.com/entry/2018/09/20/212527)

#### key_on の設定について
[python folium を使い、都道府県の夫婦年齢差をプロットする](https://www.monotalk.xyz/blog/python-folinum-%E3%82%92%E4%BD%BF%E3%81%84%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E3%81%AE%E5%A4%AB%E5%A9%A6%E5%B9%B4%E9%BD%A2%E5%B7%AE%E3%82%92%E3%83%97%E3%83%AD%E3%83%83%E3%83%88%E3%81%99%E3%82%8B/)
引用
Geojson 上に feature はキー値として存在しませんが、features は存在しており、且つ、配列です。
key_onの指定がfeature.properties.id なので、features > 1要素 feature としての解釈は folium 側でうまいことやってくれています。

今回 id としたいのは地域コードで、properties の下にぶらさがっており、on_key に feature.properties.id2 を指定しています。

In [19]:
#　一番古いH22年度のマップを作ってみます。
map_center = [36.06205900,139.66683800] #久喜市に設定
m_kuki_sh = folium.Map(location=map_center, tiles='cartodbpositron', zoom_start=13)
folium.Choropleth(geo_data=kuki_esh.to_json()).add_to(m_kuki_sh)
m_kuki_sh.choropleth(geo_data=kuki_esh.to_json(),data=df_num,
                     columns=['id', 'H22'],
                     key_on='feature.properties.id2',
                     fill_color='OrRd',
                     threshold_scale=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900],
                     fill_opacity=0.7,
                     line_opacity=0.2,
                     legend_name='生徒数'
                    )
# folium.LayerControl().add_to(m_kuki_sh)
for i, dt in df_esh.iterrows():
    folium.Marker(location=[dt['施設_緯度'],dt['施設_経度']], popup='{}'.format(dt['名称'])).add_to(m_kuki_sh)
# m_saitama
m_kuki_sh.save('kuki_esh_H22.html')